In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# get all sheet names
sheet_names = pd.ExcelFile('MAKai\MasterarbeitenDatenAlleV2.xlsx').sheet_names

dfs = []
for sheet_name in sheet_names:
    if sheet_name == 'Codierung':
        codes = pd.read_excel('MAKai\MasterarbeitenDatenAlleV2.xlsx', sheet_name=sheet_name)
    else:
        df = pd.read_excel('MAKai\MasterarbeitenDatenAlleV2.xlsx', sheet_name=sheet_name)
        df['DT'] = sheet_name
        dfs.append(df)

code_color = {
    0: 0,
    1: -1,
    2: -1,
    3: 0,
    4: 1,
    5: 0,
}

codes['code_color'] = codes['Codierung'].map(code_color)

# concatenate all dataframes
df = pd.concat(dfs, ignore_index=True)
df = df.set_index(['DT', 'Land']).melt(ignore_index=False).set_index('variable', append=True)
df = df.pivot_table(index=['Land', 'variable'], columns='DT', values='value')
df.index.names = ['Land', 'Jahr']
codes = codes.set_index('Land')
df = df.merge(codes, left_index=True, right_index=True).drop('Typ', axis=1)
df = df.set_index('Codierung', append=True).sort_index()

name_map = {
    'Kosten': ['Anteil  BIP Private', 'Anteil BIP Public ', 'Gesundheitsausgaben pro Kopf', 'Out of Pocket'],
    'Zugänglichkeit': ['Artzbesuche (pro Kopf)', 'Belegungsrate Akutpflegebet', 'Hospital beds', 'Practising doctors', 'Professional nurses'],
    'Qualität': ['Krebs M', 'Krebs W', 'Schlaganfall M', 'Schlaganfall W', 'Sterblichkeit ab 65 M', 'Sterblichkeit ab 65 W', 'Verhinderbare Sterblichkeitsrat']
}

score_map = {
    'Anteil  BIP Private': 1, 
    'Anteil BIP Public ': 1, 
    'Artzbesuche (pro Kopf)': 1,
    'Belegungsrate Akutpflegebet': -1, 
    'Gesundheitsausgaben pro Kopf': 1,
    'Hospital beds': 1, 
    'Krebs M': -1, 
    'Krebs W': -1, 
    'Out of Pocket': 1,
    'Practising doctors': 1,
    'Professional nurses': 1, 
    'Schlaganfall M': -1,
    'Schlaganfall W': -1,
    'Sterblichkeit ab 65 M': 1, 
    'Sterblichkeit ab 65 W': 1,
    'Verhinderbare Sterblichkeitsrat': -1
    }

# fill missing values
df = df.groupby(['Land']).ffill()

# z normalize over all years per variable
df = (df - df.mean()) / df.std()

# add columns for each category
for key in name_map.keys():
    tmp_sum = []
    for col in name_map[key]:
        tmp_sum.append(df[col] * score_map[col])

    tmp_sum = pd.concat(tmp_sum, axis=1).mean(axis=1)

    df[key] = tmp_sum

# filter year > 2010
df_f = df[df.index.get_level_values('Jahr') > 2010]

In [82]:
# count data points per year
df.groupby('Jahr').count().mean(axis=1)


Jahr
2006     1.25
2007    10.40
2008    24.20
2009    18.75
2010    28.95
2011    29.90
2012    30.65
2013    32.00
2014    33.15
2015    33.35
2016    34.10
2017    34.75
2018    34.20
2019    34.30
2020    35.45
2021    32.70
2022    35.90
dtype: float64

In [84]:
# plot time series grouped by DT
plt_data = df_f.groupby(['Jahr', 'Codierung']).mean()

for col in name_map.keys():
    fig = px.line(plt_data[col].unstack().ffill(), title=col)
    fig.show()

In [80]:
# 3d scatter plot
plt_data = plt_data[np.logical_and(plt_data.index.get_level_values('Codierung') != 0,plt_data.index.get_level_values('Codierung') != 3) ]
fig = px.scatter_3d(plt_data.reset_index(), x='Kosten', y='Zugänglichkeit', z='Qualität', color='code_color')
fig.update_layout(width=800, height=800)
fig.show()
